In [1]:
import numpy as np
import pandas as pd
from functions import createDataMain,createTestDataMain,pd2np,pd2np_test,createDataMainSecondStage
import cPickle

In [2]:
batch_size = 100
discreteList = ['dayOfWeek','payDay','month','earthquake',\
                'type','locale','locale_name','transferred','onpromotion']


In [3]:
with open(r"dateVar.pickle", "rb") as input_file:
    dateVar_list = cPickle.load(input_file)

Save item lookup tables

In [5]:
types = {'id': 'int32',
     'item_nbr': 'int32',
     'store_nbr': 'int8',
     'unit_sales': 'float32',
     'onpromotion': bool}
train = pd.read_csv('train.csv',usecols=['date','item_nbr','store_nbr','unit_sales','onpromotion'],\
                    parse_dates=['date'],dtype=types, infer_datetime_format=True)
train = train.fillna(2,axis=1)
train.onpromotion = train.onpromotion.astype(np.int8)
train.loc[train.unit_sales<0,'unit_sales'] = .0 # clip negative sales to zero
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
with open(r"item_dict_final.pickle", "wb") as output_file:
    cPickle.dump(item_dict, output_file)

In [7]:
train = train[train.date < '2017-07-31']
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}

In [8]:
with open(r"item_dict_train.pickle", "wb") as output_file:
    cPickle.dump(item_dict, output_file)

<br>

*** Validation Case***

In [5]:
train,val_SI,val_newItem = createDataMain(True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
train = train.sort_values('countDays').reset_index(drop=True)

In [7]:
# train[index] gives the corresponding historical Xs for test dataset
# train needs to be after sorting
index = val_SI.merge(train.reset_index()[['index','store_nbr','item_nbr']],\
                     'left',['store_nbr','item_nbr'])['index'].values.astype(np.int32)

In [8]:
np.savetxt('Index_val',index,fmt="%d",delimiter=",") 

In [9]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(val_SI,batch_size,val_SI.countDays.max(),dateVar_list,'date',discreteList)

In [15]:
prefix = 'val_SI'

In [16]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [11]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(train,batch_size,train.countDays.max(),dateVar_list,'date',discreteList)

In [12]:
prefix = 'train'

In [13]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [17]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(val_newItem,batch_size,val_newItem.countDays.max(),dateVar_list,'date',discreteList)

In [19]:
prefix = 'val_newItem'

In [20]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

Second Stage Fine Tuning

In [4]:
train,val = createDataMainSecondStage(True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
train = val.loc[:,['store_nbr','item_nbr']].merge(train,'left',['store_nbr','item_nbr'])

In [9]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(val,batch_size,val.countDays.max(),dateVar_list,'date',discreteList)
prefix = 'val_SS'
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")
    
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(train,batch_size,train.countDays.max(),dateVar_list,'date',discreteList)
prefix = 'train_SS'
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")

In [4]:
train, val = createDataMainSecondStage(False)
train = val.loc[:,['store_nbr','item_nbr']].merge(train,'left',['store_nbr','item_nbr'])

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [41]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(train,batch_size,train.countDays.max(),dateVar_list,'date',discreteList)
prefix = 'train_SS_final'
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")
    
y_np_test, Con_np_test,Dis_np_test,X_np_test = \
            pd2np_test(val,batch_size,16,dateVar_list,'date',discreteList)
y_np_test, Con_np_test = \
[np.concatenate([b[:,-1,np.newaxis],a],1) for a,b in zip([y_np_test, Con_np_test],[y_np, Con_np])]
Dis_np_test = np.concatenate([Dis_np[:,64:65,:], Dis_np_test],1)
prefix = 'test_SS_final'
np.savetxt(prefix+'_Y',y_np_test,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np_test,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np_test,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np_test[:,:,j],fmt="%d",delimiter=",")

*** Test Case ***

training dataset

In [4]:
train = createDataMain(False)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
train = train.sort_values('countDays').reset_index(drop=True)

In [6]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(train,batch_size,train.countDays.max(),dateVar_list,'date',discreteList)

In [10]:
prefix = 'train_final'

In [11]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

test dataset

In [5]:
test_SI,test_newItem = createTestDataMain(False)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
y_np, Con_np,Dis_np,X_np = \
pd2np_test(test_SI,batch_size,16,dateVar_list,'date',discreteList)

In [7]:
prefix = 'test_SI_final'

In [8]:
np.savetxt(prefix+'_Y',y_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

<br>

In [9]:
y_np, Con_np,Dis_np,X_np = \
pd2np_test(test_newItem,batch_size,16,dateVar_list,'date',discreteList)

In [10]:
prefix = 'test_newItem_final'

In [11]:
np.savetxt(prefix+'_Y',y_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [21]:
# train[index] gives the corresponding historical Xs for test dataset
# train needs to be after sorting
index = test_SI.merge(train.reset_index()[['index','store_nbr','item_nbr']],\
                     'left',['store_nbr','item_nbr'])['index'].values.astype(np.int32)

In [18]:
np.savetxt('Index_final',index,fmt="%d",delimiter=",") 